## 实验介绍

### 1.实验内容

本实验包括: 
* 学习并实现决策树算法----以ID3算法为例
* 基于决策树算法预测隐形眼镜类型

### 2.实验目标

通过本实验掌握决策树算法的基本原理。

### 3.实验知识点

* 香农熵
* 信息增益
* 决策树算法基本原理

### 4.实验环境

* python 3.6.5
* numpy 1.13.3
* matplotlib 2.2.3

### 实验准备

点击屏幕右上方的下载实验数据模块，选择下载decision_tree_glass.tgz到指定目录下，然后再依次选择点击上方的File->Open->Upload,上传刚才下载的数据集压缩包，再使用如下命令解压：

In [1]:
!tar -zxvf decision_tree_glass.tgz

decision_tree_glass/
decision_tree_glass/lenses.txt
decision_tree_glass/classifierStorage.txt


## 【海洋动物分类】

## 实验步骤：【海洋动物分类】- 概述

下表数据包含5个海洋动物，特征包括：不浮出水面是否可以生存，以及是否有脚蹼。我们将这些动物分成两类：鱼类和非鱼类。本实验要求基于决策树算法（ID3）实现对下表数据的分类。

id| 不浮出水面是否可以生存 | 是否有脚蹼 | 属于鱼类
:-: | :-: | :-:|:-:
1 | 是 | 是 | 是
2 | 是 | 是 | 是
3 | 是 | 否 | 否
4 | 否 | 是 | 否
5 | 否 | 是 | 否

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

## 实验步骤：【海洋动物分类】- 创建数据集

基于上述表格，创建数据集。

In [3]:
def createDataSet():
    """
    函数说明：创建数据集
    returns:
        dataSet - 数据集
        labels - 分类属性
    """
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing','flippers']
    return dataSet, labels

## 实验步骤：【海洋动物分类】- 计算香农熵

In [4]:
def calcShannonEnt(dataSet):
    """
    函数说明:计算给定数据集的香农熵
    parameters:
        dataSet - 数据集
    returns:
        shannonEnt - 香农熵
    """
    yes = 0
    labels = dict()
    for i in dataSet:
        if i[-1] not in labels:
            labels[i[-1]] = 1
        else:
            labels[i[-1]] += 1
    shannonEnt = 0
    for i in labels:
        shannonEnt += (-labels[i]/len(dataSet)*np.log2(labels[i]/len(dataSet)))
    return shannonEnt
    pass
data,label = createDataSet()
calcShannonEnt(data)

0.97095059445466858

In [5]:
# chooseBestFeatureToSplit(data)

## 实验步骤：【海洋动物分类】- 特征选择(基于信息增益划分数据集)

信息增益是相对于特征而言的，信息增益越大，特征对最终的分类结果影响也就越大，我们就应该选择对最终分类结果影响最大的那个特征作为我们的分类特征。

splitDataSet函数是用来选择各个特征的子集的。chooseBestFeatureToSplit函数是选择选择最优特征的函数。

In [6]:
def splitDataSet(dataSet, axis, value):
    """
    函数说明: 按照给定特征划分数据集
    parameters:
        dataSet - 待划分的数据集
        axis - 划分数据集的特征 (第axis个特征)
        value - 特征值
    returns:
        retDataSet - 划分后的数据集
    """
    retDataSet = []
    for i in dataSet:
        if i[axis] == value:
            splitdata = i[:axis]+i[axis+1:]
            retDataSet.append(splitdata)
    return retDataSet
    pass


def chooseBestFeatureToSplit(dataSet):
    """
    函数说明: 选择最优特征
    parameters:
        dataSet - 数据集
    returns:
        bestFeature - 信息增益最大的(最优)特征的索引值
    """
    infoGains = [] # 记录各个特征划分后的信息增益
    for axis in range(len(dataSet[0])-1): # 分别根据axis划分特征
        values = set([dataSet[i][axis] for i in range(len(dataSet))]) # 统计第axis个特征的所有可能的值
        newEnt = 0 # 划分后的信息熵
        for value in values:
            newdataSet = splitDataSet(dataSet,axis,value)
            newEnt += len(newdataSet)/len(dataSet)*calcShannonEnt(newdataSet)
        infoGains.append(newEnt-calcShannonEnt(dataSet))
    return infoGains.index(max(infoGains))
        
    pass

## 实验步骤：【海洋动物分类】- 构建决策树

决策树构建算法流程：得到原始数据集，基于最好的属性值划分数据集。第一次划分之后，数据将被向下传递到树分支的下一个节点，在这个节点上，再次划分数据。采用递归的原则处理数据集。

递归结束条件：程序遍历完所有划分数据集的属性，或者每个分支下的所有实例都具有相同的分类。

In [7]:
class TreeNode:
    def __init__(self,label=None,value=None):
        self.value = value
        self.label = label
        self.children = []
        self.result = None
def createTreeHelp(node,dataSet,labels):
    if len(dataSet[0]) > 2:
        axis = chooseBestFeatureToSplit(dataSet)
        values = list(set(i[axis] for i in dataSet))
        for value in values:
            newnode = TreeNode(labels[axis],value)
            node.children.append(newnode)
            nextlabels = labels[:]
            nextlabels.remove(labels[axis])
            createTreeHelp(newnode,splitDataSet(dataSet,axis,value),nextlabels)
        return node
    else:
        values = list(set(i[0] for i in dataSet))
        
        for value in values:
            dic = dict()
            for i in dataSet:
                if i[0] == value:
                    if i[1] not in dic:
                        dic[i[1]] = 1
                    else:
                        dic[i[1]] += 1
            label = list(dic.keys())[list(dic.values()).index(max(dic.values()))]
            newnode = TreeNode(labels[0],value)
            newnode.result = label
            node.children.append(newnode)

In [8]:
def createTree(dataSet, labels):
    """
    函数说明:创建决策树
    Parameters:
        dataSet - 训练数据集
        labels - 分类属性标签
    Returns:
        myTree - 决策树
    """
    root = TreeNode()
    createTreeHelp(root,dataSet,labels)
    return root
    
    pass

## 实验步骤：【海洋动物分类】- 使用决策树进行分类

In [13]:
def classify(inputTree, featLabels, testVec):
    """
    函数说明:使用决策树分类
    Parameters:
        inputTree - 已经生成的决策树
        featLabels - 存储选择的最优特征标签
        testVec - 测试数据列表，顺序对应最优特征标签
    Returns:
        classLabel - 分类结果
    """
    p = inputTree

    while p.children !=[]:
        for i in p.children:
            if i.value == testVec[featLabels.index(i.label)]:
                break
        p = i
    return p.result
    pass

In [14]:
if __name__ == '__main__':
    testVec1 = [1,0]
    testVec2 = [1,1]
    # 使用决策树对testVec1和testVec2分类
    data,label = createDataSet()
    root = createTree(data,label)
    print(classify(root,label,testVec1))
    print(classify(root,label,testVec2))

no
yes


## 【预测隐形眼镜类型】

## 实验步骤：【预测隐形眼镜类型】- 概述

本实验要求基于决策树算法，帮助人们判断需要佩戴的镜片类型。

### 数据介绍
隐形眼镜数据集是非常著名的数据集，它包含很多换着眼部状态的观察条件以及医生推荐的隐形眼镜类型。隐形眼镜类型包括硬材质(hard)、软材质(soft)以及不适合佩戴隐形眼镜(no lenses)。

数据集一共有24组数据，数据的Labels依次是age、prescript、astigmatic、tearRate、class，也就是第一列是年龄，第二列是症状，第三列是是否散光，第四列是眼泪数量，第五列是最终的分类标签。

## 实验步骤：【预测隐形眼镜类型】- 创建决策树

In [25]:
f = open('decision_tree_glass/lenses.txt','r',encoding='utf-8')
lines = f.readlines()
f.close()
glassDataSet = []
glassLabels = ['age','prescript','tearRate','class']
for line in lines:
    line = line.strip('\n').split('\t')
    glassDataSet.append(line)
# print(glassDataSet)
# print(glassLabels)
root = createTree(glassDataSet,glassLabels)


In [28]:
def pirorOrderVisit(node):
    if node.children != []:
        print(node.value,node.label)
        for i in node.children:
            pirorOrderVisit(i)

In [29]:
pirorOrderVisit(root)

None None
pre age
hyper prescript
no tearRate
yes tearRate
myope prescript
no tearRate
yes tearRate
young age
hyper prescript
no tearRate
yes tearRate
myope prescript
no tearRate
yes tearRate
presbyopic age
hyper prescript
no tearRate
yes tearRate
myope prescript
no tearRate
yes tearRate
